In [32]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pandas.io.json import json_normalize
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
import numpy as np 

In [33]:
# Unemployment_rate_df.info()

### Store CSV into DataFrame

In [34]:
# https://www.kaggle.com/jayrav13/unemployment-by-county-us#output.csv
csv_file = "Resources/Unempoyment_rate.csv"
Unemployment_rate_df = pd.read_csv(csv_file)
Unemployment_rate_df.head()

,Year,Month,State,County,Rate
0,2015,February,Mississippi,Newton County,6.1
1,2015,February,Mississippi,Panola County,9.4
2,2015,February,Mississippi,Monroe County,7.9
3,2015,February,Mississippi,Hinds County,6.1
4,2015,February,Mississippi,Kemper County,10.6


In [35]:
Unemployment_rate_split=Unemployment_rate_df['County'].str.split(" ",n=1, expand=True)
Unemployment_rate_split_df=pd.DataFrame(Unemployment_rate_split)
Unemployment_rate_split_df.columns=['county_n','dump']
Unemployment_rate_split_df.head()

,county_n,dump
0,Newton,County
1,Panola,County
2,Monroe,County
3,Hinds,County
4,Kemper,County


In [36]:
Unemployment_rate_df['county_n']=pd.Series(Unemployment_rate_split_df['county_n'])
Unemployment_rate_df.head()

,Year,Month,State,County,Rate,county_n
0,2015,February,Mississippi,Newton County,6.1,Newton
1,2015,February,Mississippi,Panola County,9.4,Panola
2,2015,February,Mississippi,Monroe County,7.9,Monroe
3,2015,February,Mississippi,Hinds County,6.1,Hinds
4,2015,February,Mississippi,Kemper County,10.6,Kemper


In [37]:
Unemployment_rate_df=Unemployment_rate_df.loc[Unemployment_rate_df['Year']==2015]
Unemployment_rate_df=Unemployment_rate_df[Unemployment_rate_df.State != 'New York']
Unemployment_rate_df=Unemployment_rate_df.rename(columns={"Month":"month","State":"state","County":"county","Rate": "monthly_rate"})
Unemployment_rate_df=Unemployment_rate_df.drop(['Year'], axis=1)
Unemployment_rate_df.drop_duplicates(subset=["state","county"], keep='first', inplace=True)
Unemployment_rate_df.reset_index(inplace=True)
Unemployment_rate_df.rename(columns={"index":"county_id"}, inplace=True)
Unemployment_rate_df.head()

,county_id,month,state,county,monthly_rate,county_n
0,0,February,Mississippi,Newton County,6.1,Newton
1,1,February,Mississippi,Panola County,9.4,Panola
2,2,February,Mississippi,Monroe County,7.9,Monroe
3,3,February,Mississippi,Hinds County,6.1,Hinds
4,4,February,Mississippi,Kemper County,10.6,Kemper


In [38]:
Unemployment_rate_df=Unemployment_rate_df.drop(['county'], axis=1)
Unemployment_rate_df=Unemployment_rate_df.rename(columns={"county_n":"county"})
Unemployment_rate_df.head()

,county_id,month,state,monthly_rate,county
0,0,February,Mississippi,6.1,Newton
1,1,February,Mississippi,9.4,Panola
2,2,February,Mississippi,7.9,Monroe
3,3,February,Mississippi,6.1,Hinds
4,4,February,Mississippi,10.6,Kemper


In [39]:
# Unemployment_rate_df.info()

In [40]:
# https://www.kaggle.com/muonneutrino/us-census-demographic-data
### Data for 2015 for Population
csv_file = "Resources/acs2015_county_data.csv"
county_pop_df = pd.read_csv(csv_file)
# county_data_df.head()

In [41]:
def find_county_id(row):
    state = row['state']
    county= row['county']
    print(f'{state} {county}')
    matched_row = Unemployment_rate_df.loc[(Unemployment_rate_df['state']== state) & (Unemployment_rate_df['county']==county)]
    return matched_row['county_id']
county_pop_df.apply(find_county_id, axis=1)

In [42]:
###Create new data with select columns
### Data for 2015
county_col=["State","County","TotalPop","Men","Women"]
county_pop_df = county_pop_df[county_col].copy()

# Set index
# county_transformed.set_index("State", inplace=True)
county_pop_df=county_pop_df.rename(columns={"State":"state","County":"county","TotalPop":"totalpop","Men":"men","Women":"women"})
county_pop_df.reset_index()
county_pop_df.drop_duplicates(subset=["county"], keep='first', inplace=True)

county_pop_df.reset_index(inplace=True)
county_pop_df.rename(columns={"index":"id"}, inplace=True)

county_pop_df.head()

,id,state,county,totalpop,men,women
0,0,Alabama,Autauga,55221,26745,28476
1,1,Alabama,Baldwin,195121,95314,99807
2,2,Alabama,Barbour,26932,14497,12435
3,3,Alabama,Bibb,22604,12073,10531
4,4,Alabama,Blount,57710,28512,29198


In [43]:
# county_pop_df.iloc[0]

In [44]:
# find_county_id(county_pop_df.iloc[0])

In [45]:
# county_pop_df['county_id']=county_pop_df.apply(np.find_county_id, axis=0)
# county_pop_df = county_pop_df.apply(lambda row : replace(row)) 


## Store JSON data into a DataFrame

In [48]:
json_file = "Resources/Unempoyment_yearlry_rate.json"
# print(json_file)

In [49]:
filepath = json_file
with open(filepath) as jsonfile:
    json_norm = json.load(jsonfile)
# print(json_norm)

In [50]:
# json_normalize(json_norm)
# print(json.dumps(json_norm, indent=4))


In [51]:
json_2015 = json_norm['2015']['Annual']
# print(json.dumps(json_2015, indent=4))



In [52]:
data={}
state_list=[]
for state in json_2015:
    
    data[state]=[]
    for county in json_2015[str(state)]['Unemployment Rate'].items():
        data[state].append(county)
        
for state in json_2015:
    state_list.append(state)
    
print( data[state])

[('Albany County', 3.1), ('Big Horn County', 4.3), ('Campbell County', 3.8), ('Carbon County', 3.6), ('Converse County', 3.6), ('Crook County', 3.4), ('Fremont County', 5.4), ('Goshen County', 3.2), ('Hot Springs County', 4.1), ('Johnson County', 4.8), ('Laramie County', 3.9), ('Lincoln County', 4.6), ('Natrona County', 4.8), ('Niobrara County', 2.9), ('Park County', 4.2), ('Platte County', 3.8), ('Sheridan County', 4.1), ('Sublette County', 5.1), ('Sweetwater County', 4.6), ('Teton County', 3.8), ('Uinta County', 4.9), ('Washakie County', 4.0), ('Weston County', 3.3)]


In [53]:
annual_rate_df=pd.DataFrame(data[state], columns=('County','Annual_rate'))
annual_rate_df=annual_rate_df.rename(columns={"County":"county","Annual_rate":"annual_rate"})
annual_rate_df.reset_index()
annual_rate_df.drop_duplicates(subset=["county"], keep='first', inplace=True)

annual_rate_df.reset_index(inplace=True)
annual_rate_df.rename(columns={"index":"county_id"}, inplace=True)

annual_rate_df.head()

,county_id,county,annual_rate
0,0,Albany County,3.1
1,1,Big Horn County,4.3
2,2,Campbell County,3.8
3,3,Carbon County,3.6
4,4,Converse County,3.6


### Connect to database

In [25]:
connection_string = "postgres:artinaysa@localhost:5432/ETL_db"
engine = create_engine(f'postgresql://{connection_string}')

In [26]:
# Confirm tables
engine.table_names()

['unemployment_rate', 'county_pop', 'annual_rate']

### Load DataFrame into database

In [27]:
Unemployment_rate_df.to_sql(name='unemployment_rate', con=engine, if_exists='append', index=False)

In [46]:
county_pop_df.to_sql(name='county_pop', con=engine, if_exists='append', index=False)

In [54]:
annual_rate_df.to_sql(name='annual_rate', con=engine, if_exists='append', index=False)

### Confirm data has been added to database

In [55]:
pd.read_sql_query('select * from unemployment_rate', con=engine).head()

,county_id,month,state,county,monthly_rate
0,0,February,Mississippi,Newton County,6
1,1,February,Mississippi,Panola County,9
2,2,February,Mississippi,Monroe County,8
3,3,February,Mississippi,Hinds County,6
4,4,February,Mississippi,Kemper County,11


In [56]:
pd.read_sql_query('select * from county_pop', con=engine).head()

,id,state,county,totalpop,men,women
0,0,Alabama,Autauga,55221,26745,28476
1,1,Alabama,Baldwin,195121,95314,99807
2,2,Alabama,Barbour,26932,14497,12435
3,3,Alabama,Bibb,22604,12073,10531
4,4,Alabama,Blount,57710,28512,29198


In [57]:
pd.read_sql_query('select * from annual_rate', con=engine).head()

,county_id,county,annual_rate
0,0,Albany County,3
1,1,Big Horn County,4
2,2,Campbell County,4
3,3,Carbon County,4
4,4,Converse County,4
